# Measuring the effectiveness Few-shot learning for traffic sign recognition

# ABSTRACT
Nowadays, self-driving veichles is one of the main application of Artificial Intelligence (AI) which has been most widely used in daily-life scenarios. These veichles are based on many AI models among which there are traffic sign recognition systems. Different proposals have been provided for these systems, most of which rely on the usage of big datasets such as GTSRB (dataset of German traffic signs). Collecting a big amount of data is one of the main problems of Machine Learning models, which need lots of evidences in order to being able to generalize well. The effort in collecting more data than available can be one of the main aspects to consider during the development of these systems. Furthermore, using a big dataset expose to possible Adversarial attacks, from poisoning to evasion ones, specially in case of out-source training and testing. For these reasons, in this work we investigate the application of state-of-art Few-shot learning models on the traffic sign recognition domain. The aim of this work is to define the effectiveness of these approaches, in contrast to non-few-shot ones, to open the possibilities of build more robust and effortless solutions in this field.

# NOTE
più peso alla motivazione dell'attacco rispetto alla difficoltà nella raccolta di dati. Quindi focalizzarsi sul problema che attacchi di poisoning necessitano di aggiungere esempi poisonati che sono facilmente scopribili in casi di dataset piccoli (come nel few-shot) mediante un controllo umano dei sample. Attenzione: verificare, in caso questo controllo non si possa fare, se l'attacco è avvantaggiato con un dataset più piccolo rispetto ad uno grande. Se cosi fosse, sarebbe un problema.

in the related work there are the works we are trying to improve

specificare come ho creato il dataset ridotto

testare i modelli anche sul dataset ridotto

cercare come vengono valutati i modelli di few shot. Se si usa bootstrap o altre tecniche

usare più modelli few shot, non solo CLIP e LDC

-  N-way-k-shot (it means N classes and K instances per classes)

-  Approfondire discorso support set (set non utilizzato nè durante training nè durante test ma solo durante fase di predizione)


# RICERCHE ESISTENTI
- <b>FUSED-Net: Enhancing Few-Shot Traffic Sign Detection with Unfrozen Parameters, Pseudo-Support Sets, Embedding Normalization, and Domain Adaptation</b>: https://arxiv.org/html/2409.14852v1 SOLO SU DATASET DEL BANGLADESH

- <b>Cross-domain Few-shot In-context Learning for Enhancing Traffic Sign Recognition</b>: questo fa un compare tra modelli few shot sul dataset GTSRB

- <b>Few-shot traffic sign recognition with clustering inductive bias and random neural network</b>: questo propone un nuovo modello per few-shot su traffic sign recognition

- <b>Meta-YOLO: Meta-Learning for Few-Shot Traffic Sign Detection via Decoupling Dependencies</b>: questo fa traffic sign detection e non recognition (usa YOLO) quindi a noi non ci interessa

- <b>Research on a Traffic Sign Recognition Method under Small Sample Conditions</b>: nuovo modello proposto

- <b>Self-supervised few-shot learning for real-time traffic sign classification</b>: altro modello introdotto

# EVALUATION METRICS

Possible evaluation metrics used in the papers above are:
Ma precision e recall non sono per binary classification? Lo dovresti fare per ogni classe considerando tutte le altre classi come istanze negative?
- Average precision (AP): area under the precision-recall curve
- Precision
- Recall
- A sto punto anche F1 Score? Male non fa
- Accuracy rate (consigliato al posto di error rate. più utilizzato in letteratura)
-

# EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS
- <b>Bootstrap</b>
- <b>Fine-Grained Image Classification</b>: si valutano e testano gruppi di classi alla volta in modo tale da identificare i gruppi più problematici (nel nostro caso il gruppo dei segnali stradali relativi ai limiti di velocità potrebbero essere piu difficili da classificare rispetto a quelli di pericolo). "Fine-Grained Image Classification is a task in computer vision where the goal is to classify images into subcategories within a larger category. For example, classifying different species of birds or different types of flowers. This task is considered to be fine-grained because it requires the model to distinguish between subtle differences in visual appearance and patterns, making it more challenging than regular image classification tasks."(https://paperswithcode.com/task/fine-grained-image-classification)
-<b>Generalization error</b>: How much does accuracy drop when we reduce K (e.g., from K=5 to K=1)?*


# EVALUATION TECHNIQUES TO COMPARE THE FEW SHOT VS MANY-SHOT LEARNING
- <b>T-test</b> Separiamo il dataset di test in più dataset dove ciascun dataset contiene i sample di un determinato gruppo di segnali stradali (quindi un gruppo di classi) (vedere Fine-Grained Image Classification in  EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS). Siccome i dataset sono indipendenti gli uni dagli altri possiamo tranquillamente applicare il t-test classico per vedere se l'accuracy rate dell'algoritmo few shot e l'accuracy rate dell'algoritmo non-few shot sono significativamente diversi.





# Nuova sezione

# Nuova sezione

In [1]:
!git clone https://github.com/mattiacolucci/few-shot-traffic-sign

Cloning into 'few-shot-traffic-sign'...
remote: Enumerating objects: 3857, done.
remote: Counting objects: 100% (3857/3857), done.
remote: Compressing objects: 100% (3842/3842), done.
remote: Total 3857 (delta 18), reused 3845 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (3857/3857), 10.28 MiB | 27.06 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
import importlib.util
import sys

sys.path.append("./few-shot-traffic-sign/")


spec = importlib.util.spec_from_file_location("__main__", "./few-shot-traffic-sign/ldc_train.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["__main__"] = foo
spec.loader.exec_module(foo)
foo.MyClass()

ModuleNotFoundError: No module named 'ftfy'